In [2]:
import numpy as np

In [3]:
# Función para calcular la norma \(p\) de una matriz \(x\) por filas
def lp_matrix(x: np.ndarray, p=1) -> float:
    if p == "inf":
        return np.max(np.abs(x), axis=1)
    return np.sum(np.abs(x)**p, axis=1)**(1/p)

# Función para calcular la norma \(p\) de un vector \(x\)
def lp_vector(x: np.ndarray, p=1) -> float:
    if p == "inf":
        return np.max(np.abs(x))
    return np.sum(np.abs(x)**p)**(1/p)

In [4]:
# Función que aproxima la norma \(p\) de una matriz \(A\) mediante muestreo Monte Carlo
def numeric_approach_lp_matrix_norm(matrix, p, samples):
    dimension = len(matrix)
    montecarlo = np.random.random((samples, dimension))
    
    # Normalizamos los vectores de Montecarlo
    montecarlo_norm = montecarlo / lp_matrix(montecarlo, p).reshape(-1, 1)

    # Calculamos la norma inducida
    norm_induced = lp_matrix(np.dot(matrix, montecarlo_norm.T).T, p)
    idx = np.argmax(norm_induced)

    # Comparamos con el valor real de la norma \(p\) de la matriz \(A\)
    if p != "inf":
        print("Numeric approach: ", lp_vector(np.dot(matrix, montecarlo_norm[idx, :]), p), " Vs. Real value: ", np.linalg.norm(matrix, ord=p))
    else:
        print("Numeric approach: ", lp_vector(np.dot(matrix, montecarlo_norm[idx, :]), p), " Vs. Real value: ", np.linalg.norm(matrix, ord=np.inf))
    
    return montecarlo_norm[idx, :]

In [5]:
# Función que aproxima la norma (p) de una matriz (A) mediante un enfoque iterativo
def iterative_numeric_approach_lp_matrix_norm(A, p, samples, max_iter=500):
    dimension = len(A)
    
    if p != "inf":
        real_val = np.linalg.norm(A, ord=p)
    else:
        real_val = np.linalg.norm(A, ord=np.inf)
    
    montecarlo = np.random.random((samples*10, dimension))

    # Normalizamos los vectores de Montecarlo
    montecarlo_norm = montecarlo / lp_matrix(montecarlo, p).reshape(-1, 1)

    # Calculamos la norma inducida inicial
    norm_induced = lp_matrix(np.dot(A, montecarlo_norm.T).T, p)
    idx = (-norm_induced).argsort()[:2]

    vector_a = montecarlo_norm[idx[0], :]
    vector_b = montecarlo_norm[idx[1], :]

    for j in range(max_iter):
        points_alpha = np.random.random((dimension, samples))
        points_beta = np.random.random((dimension, samples))

        # Combinamos los vectores alpha y beta
        combination = np.array([np.multiply(points_alpha[:, i], vector_a) + np.multiply(1-points_alpha[:, i], vector_b) for i in range(samples)]).reshape(-1, dimension)
        combination_norm = combination / lp_matrix(combination, p).reshape(-1, 1)

        # Calculamos la norma inducida con la combinación
        norm_induced = lp_matrix(np.dot(A, combination_norm.T), p)
        idx = (-norm_induced).argsort()[:2]

        # Si la diferencia es pequeña, consideramos que hemos convergido
        if np.abs(lp_vector(np.dot(A, combination_norm[idx[0], :]), p) - real_val) < 0.01:
            print(j)
            print("Numeric approach: ", lp_vector(np.dot(A, combination_norm[idx[0], :]), p), " Vs. Real value: ", real_val)
            return combination_norm[idx[0], :]
        else:
            if np.abs(lp_vector(np.dot(A, vector_a), p) - real_val) < np.abs(lp_vector(np.dot(A, combination_norm[idx[0], :]), p) - real_val):
                vector_a = combination_norm[idx[0], :]
                vector_b = combination_norm[idx[1], :]

    print(j)
    print("Numeric approach: ", lp_vector(np.dot(A, combination_norm[idx[0], :]), p), " Vs. Real value: ", real_val)
    return combination_norm[idx[0], :]


In [6]:
A = np.random.randint(-10,10,(5,5))
p = 2
print(A)

print("vector method 1: ", numeric_approach_lp_matrix_norm(A, p, 1000))

print("Vector method iterative: ", iterative_numeric_approach_lp_matrix_norm(A, p, 1000, 1000))

[[ 7 -8  3  9  7]
 [ 3 -4  6  1  3]
 [ 6 -8  3  3 -2]
 [-4  4 -8 -4 -3]
 [-2 -6  2  7  4]]
Numeric approach:  18.686426644424806  Vs. Real value:  23.217533446013622
vector method 1:  [0.29438254 0.00558819 0.56571753 0.65113849 0.41144871]
999
Numeric approach:  18.983754351293605  Vs. Real value:  23.217533446013622
Vector method iterative:  [0.41634562 0.00402346 0.5155937  0.62216042 0.41679695]


Usando el algoritmo del ítem anterior estime la norma inducida de
ordenes 1, 2, 3 y ∞ para la matríz de Hilbert de tamaño 10 × 10, definida por $H_{10} = [h_{ij}] = \frac{1}{i+j}$.

In [18]:
#Matriz de Hilbert de orden 10
H = np.array([[1/(i+j+1) for j in range(10)] for i in range(10)])

#Estimacion norma 1
p = 1

print("vector method 1: ", numeric_approach_lp_matrix_norm(H, p, 1000))

print("Vector method iterative: ", iterative_numeric_approach_lp_matrix_norm(H, p, 1000, 1000))

Numeric approach:  1.7292343579570306  Vs. Real value:  2.9289682539682538
vector method 1:  [0.22993203 0.19933105 0.10165035 0.13468446 0.09434681 0.01157474
 0.03810378 0.05782165 0.11783053 0.0147246 ]
999
Numeric approach:  1.9305166925267283  Vs. Real value:  2.9289682539682538
Vector method iterative:  [0.37105347 0.13422846 0.1362357  0.05320778 0.07499798 0.01038229
 0.06458771 0.06514294 0.06971148 0.02045219]


In [19]:
#Estimacion norma 2
p = 2

print("vector method 1: ", numeric_approach_lp_matrix_norm(H, p, 1000))

print("Vector method iterative: ", iterative_numeric_approach_lp_matrix_norm(H, p, 1000, 1000))

Numeric approach:  1.6878197299027358  Vs. Real value:  1.7519196702651776
vector method 1:  [0.58385264 0.51158944 0.36261358 0.16969961 0.27545323 0.09057298
 0.08373139 0.21059926 0.19944526 0.24876981]
999
Numeric approach:  1.7021909459662727  Vs. Real value:  1.7519196702651776
Vector method iterative:  [0.61321693 0.56004294 0.29025925 0.20674626 0.12634658 0.1872607
 0.24912564 0.23704849 0.10452766 0.05577478]


In [21]:
#Estimacion norma infinita
p = "inf"

print("vector method 1: ", numeric_approach_lp_matrix_norm(H, p, 1000))

print("Vector method iterative: ", iterative_numeric_approach_lp_matrix_norm(H, p, 1000, 1000))

Numeric approach:  2.5411736652410606  Vs. Real value:  2.9289682539682538
vector method 1:  [0.99409425 1.         0.62519041 0.27404397 0.8919944  0.96473979
 0.98541839 0.70196582 0.94692265 0.97249366]
999
Numeric approach:  2.51128058421513  Vs. Real value:  2.9289682539682538
Vector method iterative:  [1.         0.92640483 0.98506107 0.97287308 0.61374259 0.13852289
 0.92632303 0.82638635 0.46220708 0.43684043]
